In [2]:
#The threading library can be used to execute any Python callable in 
#its own thread. To do this, you create a Thread instance and supply 
#the callable that you wish to execute as a target. Here is a simple example:

In [3]:
# code to execute in an independent thread
import time
def countdown(n):
    while n>0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

In [9]:
# create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(10,))
t.start()

T-minus 10


In [10]:
# Threads are executed in their own system-level 
# thread (e.g., a POSIX thread or Windows threads)
# that is fully managed by the host operating system. 
# Once started, threads run independently until the 
# target function returns. You can query a thread 
# instance to see if it’s still running:

In [11]:
if t.is_alive():
    print('Still running')
else:
    print('Completed')

Still running
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2


In [12]:
# You can also request to join with a thread, which waits 
# for it to terminate:

T-minus 1


In [13]:
t.join()

In [14]:
# The interpreter remains running until all threads terminate. 
# For long-running threads or background tasks that run forever, 
# you should consider making the thread daemonic. For example:

In [15]:
t = Thread(target=countdown, args=(10,), daemon=True)
t.start()

T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4


In [16]:
# Daemonic threads can’t be joined. However, they are destroyed 
# automatically when the main thread terminates.

T-minus 3
T-minus 2
T-minus 1


In [18]:
# If you want to be able to terminate threads, the thread must be 
# programmed to poll for exit at selected points. For example, you
# might put your thread in a class such as this:

In [20]:
class CountdownTask:
    def __init__(self):
        self._running = True
    def terminate(self):
        self._running = False
    def run(self, n):
        while self._running and n>0:
            print('T-minus', n)
            n -= 1
            time.sleep(5)

In [21]:
c = CountdownTask()
t = Thread(target=c.run, args=(10,))
t.start()
c.terminate()
t.join()

T-minus 10


In [ ]:
# Polling for thread termination can be tricky to coordinate 
# if threads perform blocking

In [22]:
class IOTask:
    def terminate(self):
        self._running = False
    def run(self, sock):
        # sock is a socket
        sock.settimeout(5)
        while self._running:
            # Perform a blocking I/O operation w/ timeout
            try:
                data = sock.recv(8192)
                break
            except socket.timeout:
                continue
            # Continued processing
        # Terminated
        return